In [ ]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

#######################################################################################

# our neural net class
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc1 = nn.Linear(2,1,True)
    def forward(self, x):
        x = self.fc1(x)
        return x
    
# make the neuron
net = Perceptron()
# net.cuda() # GPU acceleration

#######################################################################################

# make some synthetic data
N = 100
Nhalf = int( N / 2 )
data = torch.randn(N,2) 
for i in range(Nhalf): # class 1
    data[i,0] = data[i,0] * 0.1 + 0.2
    data[i,1] = data[i,1] * 0.1 + 0.2
for i in range(Nhalf): # class 2
    data[i+Nhalf,0] = data[i+Nhalf,0] * 0.1 + -0.2
    data[i+Nhalf,1] = data[i+Nhalf,1] * 0.1 + 0.8
    
# labels
L = torch.ones(N)
for i in range(Nhalf): # class 1
    L[i] = 1
for i in range(Nhalf): # class 2
    L[i+Nhalf] = -1
        
class Dataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        'Initialization'
        self.labels = labels
        self.data = data
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)
  def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        X = self.data[index,:]
        y = self.labels[index]
        return X, y   
    
# pack our data into a class    
train_data = Dataset(data,L)    
# this is the loader we will use to sample from below
td = torch.utils.data.DataLoader(train_data,batch_size=1,shuffle=False)    
    
#######################################################################################

from torch.utils.tensorboard import SummaryWriter
import datetime, os

%load_ext tensorboard
#%reload_ext tensorboard

writer = SummaryWriter('NN/runs')

#######################################################################################

# criteria fx
def criterion(out,label):
    return (label - out)**2

# setup SGD optimization
optimizer = optim.SGD(net.parameters(), lr=0.01)

# train
for epoch in range(250):
    accum=0
    for sample, label in td:
        outputs = net(sample)             # this passes in a set of data
        loss = criterion(outputs, label)  # forward pass on set of data
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
        accum=accum+loss
    writer.add_scalar('loss', torch.sum(accum)/N, epoch) # post that to our file for TensorBoard to analyze
        
#######################################################################################  
writer.close()

#%tensorboard --logdir runs
%tensorboard --logdir='./NN/runs'